In [20]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [22]:
#read clean dataset
filename = "/Users/vigadam/Documents/github/renfe-analysis/data/1_routes/MAD_SEV_all.pkl"

data = pd.read_pickle(filename).reset_index(drop=True)

In [24]:
data.loc[30030, "meta"]

'{"Turista": {"Promo": 47.8, "Promo +": 51.65, "Flexible": 77.1}, "Preferente": {"Promo": 80.55, "Promo +": 87.05, "Flexible": 129.9}}'

In [25]:
# generate train_id
data["train_id"] = data[["origin", "destination", "departure","arrival"]].sum(axis=1)

In [26]:
data["weekday"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()
)
data["depart_month"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month
)
data["depart_hour"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour
)
# only jan, feb, march,
data = data.loc[data["depart_month"] < 4].reset_index(drop=True)

In [27]:
data["depart_year"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").year
)

In [28]:
data["depart_year"].value_counts()

2020    284175
Name: depart_year, dtype: int64

In [29]:
len(data["train_id"].unique())

973

In [30]:
# a price-seat adatok a meta változóban vannak,
# ami egy mindig más struktúrájú dictionary,
# elég trükkös volt kinyerni, arra írtam ezt a fgv-t


def extract_meta(train):
    prices_seats = pd.DataFrame()
    for i in tqdm(range(train.shape[0])):
        dic_in = json.loads(train.meta[i])
        # extract values from tree dictionary
        while type(list(dic_in.values())[0]) != float:
            res = {key: list(value) for key, value in dic_in.items()}
            dic_out = {}
            for key in res.keys():
                for value in res[key]:
                    dic_out[key + "_" + value] = dic_in[key][value]
            dic_in = dic_out

        # check for seats, if no seat its the price
        for key in list(dic_in.keys()):
            if "seats" not in key:
                if "price" not in key:
                    dic_in[key + "_price"] = dic_in[key]
                    dic_in.pop(key)

        prices_seats = pd.concat(
            [prices_seats, pd.DataFrame.from_dict(dic_in, orient="index").T]
        ).reset_index(drop=True)
    return prices_seats

In [31]:
(data.shape[0]//10000)*10000

280000

In [34]:
chunks = {
    i: pd.concat(
        [
            data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True),
            extract_meta(
                data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True)
            ),
        ],
        axis=1,
    )
    for i in range(data.shape[0]//10000)
}

100%|██████████| 10000/10000 [00:38<00:00, 261.76it/s]


In [35]:
df = pd.concat(list(chunks.values()))

In [36]:
last_chunk = pd.concat(
    [
        data.loc[(data.shape[0]//10000)*10000:, :].reset_index(drop=True),
        extract_meta(data.loc[(data.shape[0]//10000)*10000:, :].reset_index(drop=True)),
    ]
)

100%|██████████| 4175/4175 [00:15<00:00, 277.25it/s]


In [37]:
df = pd.concat([df,last_chunk])

In [38]:
df = df.dropna(subset=["id"]).reset_index(drop=True)

In [39]:
df.columns

Index(['id', 'company', 'origin', 'destination', 'departure', 'arrival',
       'duration', 'vehicle_type', 'vehicle_class', 'price', 'fare', 'seats',
       'meta', 'insert_date', 'train_id', 'weekday', 'depart_month',
       'depart_hour', 'depart_year', 'Turista_Promo_price',
       'Turista_Promo +_price', 'Turista_Flexible_price',
       'Preferente_Promo_price', 'Preferente_Promo +_price',
       'Preferente_Flexible_price', 'Turista_Mesa_price',
       'Preferente_Mesa_price', 'Turista Plus_Flexible_price',
       'Turista_Promo_seats', 'Turista_Promo +_seats',
       'Turista_Flexible_seats', 'Preferente_Promo_seats',
       'Preferente_Promo +_seats', 'Preferente_Flexible_seats',
       'Preferente_Mesa_seats', 'Turista_Mesa_seats',
       'Turista Plus_Flexible_seats', 'Turista Plus_Promo_price',
       'Turista Plus_Promo_seats', 'Turista Plus_Promo +_price',
       'Turista Plus_Promo +_seats', 'Preferente_YOVOY VERANO_price',
       'Preferente_YOVOY VERANO_seats'],
      

In [40]:
df["days_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    ).days
    for i in range(df.shape[0])
]

df["timedelta_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    )
    for i in range(df.shape[0])
]

In [41]:
df.to_pickle("/Users/vigadam/Documents/github/renfe-analysis/data/2_routes_excluded/MAD_SEV_trains.pkl")

___